# 00 - System Information

This notebook displays system resources and VASP configuration.
Run this first to verify your environment is set up correctly.

In [1]:
import os
import subprocess

print("=" * 60)
print("System Information")
print("=" * 60)
print()

System Information



## CPU Information

In [2]:
print("CPU Information")
print("-" * 40)

# Number of CPUs
cpu_count = os.cpu_count()
print(f"  CPU cores available: {cpu_count}")

# Try to get CPU model
try:
    with open('/proc/cpuinfo', 'r') as f:
        for line in f:
            if 'model name' in line:
                cpu_model = line.split(':')[1].strip()
                print(f"  CPU model: {cpu_model}")
                break
except FileNotFoundError:
    pass

print()

CPU Information
----------------------------------------
  CPU cores available: 56
  CPU model: Intel(R) Xeon(R) CPU E5-2680 v4 @ 2.40GHz



## Memory Information

In [3]:
print("Memory Information")
print("-" * 40)

try:
    with open('/proc/meminfo', 'r') as f:
        meminfo = {}
        for line in f:
            parts = line.split(':')
            if len(parts) == 2:
                key = parts[0].strip()
                value = parts[1].strip().split()[0]  # Get numeric part
                meminfo[key] = int(value)
    
    total_gb = meminfo.get('MemTotal', 0) / 1024 / 1024
    avail_gb = meminfo.get('MemAvailable', 0) / 1024 / 1024
    
    print(f"  Total memory: {total_gb:.1f} GB")
    print(f"  Available memory: {avail_gb:.1f} GB")
except FileNotFoundError:
    print("  Unable to read memory info")

print()

Memory Information
----------------------------------------
  Total memory: 125.8 GB
  Available memory: 102.3 GB



## GPU Information

In [4]:
print("GPU Information")
print("-" * 40)

# Check for NVIDIA GPUs
try:
    result = subprocess.run(
        ['nvidia-smi', '--query-gpu=name,memory.total,driver_version', '--format=csv,noheader'],
        capture_output=True, text=True, timeout=5
    )
    if result.returncode == 0 and result.stdout.strip():
        gpus = result.stdout.strip().split('\n')
        print(f"  NVIDIA GPUs found: {len(gpus)}")
        for i, gpu in enumerate(gpus):
            parts = [p.strip() for p in gpu.split(',')]
            print(f"    GPU {i}: {parts[0]}")
            if len(parts) > 1:
                print(f"           Memory: {parts[1]}")
    else:
        print("  No NVIDIA GPUs detected")
except (FileNotFoundError, subprocess.TimeoutExpired):
    print("  nvidia-smi not available (no NVIDIA GPU or drivers)")

# Check for AMD GPUs
try:
    result = subprocess.run(
        ['rocm-smi', '--showproductname'],
        capture_output=True, text=True, timeout=5
    )
    if result.returncode == 0 and 'GPU' in result.stdout:
        print(f"  AMD ROCm GPUs detected")
        for line in result.stdout.split('\n'):
            if 'GPU' in line:
                print(f"    {line.strip()}")
except (FileNotFoundError, subprocess.TimeoutExpired):
    pass  # ROCm not installed

print()

GPU Information
----------------------------------------
  nvidia-smi not available (no NVIDIA GPU or drivers)



## VASP Environment Variables

In [5]:
print("VASP Environment Variables")
print("-" * 40)

vasp_vars = [
    ('VASP_PP_PATH', 'Pseudopotential directory'),
    ('VASP_COMMAND', 'VASP executable command'),
    ('ASE_VASP_COMMAND', 'ASE VASP command (legacy)'),
    ('VASP_SCRIPT', 'VASP submission script'),
    ('VASP_MPI_COMMAND', 'MPI launcher command'),
]

for var, description in vasp_vars:
    value = os.environ.get(var)
    if value:
        print(f"  {var}:")
        print(f"    {value}")
    else:
        print(f"  {var}: (not set)")

print()

VASP Environment Variables
----------------------------------------
  VASP_PP_PATH:
    /opt/vasp_pp/5.4
  VASP_COMMAND:
    mpirun -np 8 --map-by hwthread /opt/vasp.6.1.2_pgi_mkl_beef/bin/vasp_std
  ASE_VASP_COMMAND: (not set)
  VASP_SCRIPT: (not set)
  VASP_MPI_COMMAND: (not set)



## Pseudopotential Check

In [6]:
print("Pseudopotential Library")
print("-" * 40)

pp_path = os.environ.get('VASP_PP_PATH')

if pp_path and os.path.isdir(pp_path):
    print(f"  Path: {pp_path}")
    
    # Check for common PP directories
    pp_types = ['potpaw_PBE', 'potpaw_LDA', 'potpaw_GGA', 'potUSPP_LDA', 'potUSPP_GGA']
    found = []
    
    for pp in pp_types:
        if os.path.isdir(os.path.join(pp_path, pp)):
            found.append(pp)
    
    if found:
        print(f"  Available PP sets:")
        for pp in found:
            # Count elements
            pp_dir = os.path.join(pp_path, pp)
            try:
                elements = len([d for d in os.listdir(pp_dir) if os.path.isdir(os.path.join(pp_dir, d))])
                print(f"    - {pp} ({elements} elements)")
            except PermissionError:
                print(f"    - {pp}")
    else:
        print("  No standard PP directories found")
        print(f"  Contents: {os.listdir(pp_path)[:5]}...")
else:
    print("  VASP_PP_PATH not set or invalid")
    print("  Set this variable to your pseudopotential directory")

print()

Pseudopotential Library
----------------------------------------
  Path: /opt/vasp_pp/5.4
  Available PP sets:
    - potpaw_PBE (327 elements)
    - potpaw_LDA (295 elements)



## Python Environment

In [7]:
import sys

print("Python Environment")
print("-" * 40)
print(f"  Python version: {sys.version.split()[0]}")
print(f"  Python path: {sys.executable}")
print()

# Check key packages
packages = ['ase', 'numpy', 'scipy', 'vasp']
print("  Key packages:")
for pkg in packages:
    try:
        mod = __import__(pkg)
        version = getattr(mod, '__version__', 'unknown')
        print(f"    {pkg}: {version}")
    except ImportError:
        print(f"    {pkg}: NOT INSTALLED")

print()

Python Environment
----------------------------------------
  Python version: 3.9.15
  Python path: /opt/conda/bin/python

  Key packages:
    ase: 3.22.1
    numpy: 1.23.5
    scipy: 1.10.0
    vasp: 2.0.0



## Summary

In [8]:
print("=" * 60)
print("Environment Check Summary")
print("=" * 60)
print()

# Check readiness
checks = []

# VASP_PP_PATH
pp_ok = pp_path and os.path.isdir(pp_path)
checks.append(('VASP_PP_PATH set', pp_ok))

# VASP command
vasp_cmd = os.environ.get('VASP_COMMAND') or os.environ.get('ASE_VASP_COMMAND')
checks.append(('VASP command configured', bool(vasp_cmd)))

# ASE installed
try:
    import ase
    checks.append(('ASE installed', True))
except ImportError:
    checks.append(('ASE installed', False))

# vasp package
try:
    import vasp
    checks.append(('vasp-ase package installed', True))
except ImportError:
    checks.append(('vasp-ase package installed', False))

for check, passed in checks:
    status = '\u2714' if passed else '\u2718'
    print(f"  {status} {check}")

print()

if all(passed for _, passed in checks):
    print("All checks passed! Ready to run VASP calculations.")
else:
    print("Some checks failed. Review the configuration above.")

print()
print("Next: Run 01_getting_started.ipynb to test a simple calculation.")

Environment Check Summary

  ✔ VASP_PP_PATH set
  ✔ VASP command configured
  ✔ ASE installed
  ✔ vasp-ase package installed

All checks passed! Ready to run VASP calculations.

Next: Run 01_getting_started.ipynb to test a simple calculation.
